In [1]:
from ants import * 

In [2]:
class Ant(Insect):
    """An Ant occupies a place and does work for the colony."""

    implemented = False  # Only implemented Ant classes should be instantiated
    food_cost = 0
    is_container = False
    # ADD CLASS ATTRIBUTES HERE

    def __init__(self, health=1):
        """Create an Insect with a HEALTH quantity."""
        super().__init__(health)

    @classmethod
    def construct(cls, gamestate):
        """Create an Ant for a given GameState, or return None if not possible."""
        if cls.food_cost > gamestate.food:
            print('Not enough food remains to place ' + cls.__name__)
            return
        return cls()

    def can_contain(self, other):
        return False

    def store_ant(self, other):
        assert False, "{0} cannot contain an ant".format(self)

    def remove_ant(self, other):
        assert False, "{0} cannot contain an ant".format(self)

    def add_to(self, place):
        if place.ant is None:
            place.ant = self
        # else:
        #     # BEGIN Problem 8
        #     assert place.ant is None, 'Two ants in {0}'.format(place)
        #     # END Problem 8
        else:
            assert (
                (place.ant is None)
                or self.can_contain(place.ant)
                or place.ant.can_contain(self)
            ), 'Two ants in {0}'.format(place)
            if place.ant.is_container and place.ant.can_contain(self):
                place.ant.store_ant(self)
            elif self.is_container and self.can_contain(place.ant):
                self.store_ant(place.ant)
                # the place.ant should refer to
        
        Insect.add_to(self, place)

    def remove_from(self, place):
        if place.ant is self:
            place.ant = None
        elif place.ant is None:
            assert False, '{0} is not in {1}'.format(self, place)
        else:
            place.ant.remove_ant(self)
        Insect.remove_from(self, place)

    def double(self):
        """Double this ants's damage, if it has not already been doubled."""
        # BEGIN Problem 12
        "*** YOUR CODE HERE ***"
        # END Problem 12

# Problem 6
## Unlock Questions
### Q1
Q: What class does WallAnt inherit from?
Choose the number of the correct choice:
0) The WallAnt class does not inherit from any class
1) ThrowerAnt
2) Ant
3) HungryAnt

A: 2

### Q2
Q: What is a WallAnt's action?
Choose the number of the correct choice:
* 0) A WallAnt attacks all the Bees in its place each turn
* 1) A WallAnt increases its own health by 1 each turn
* 2) A WallAnt reduces its own health by 1 each turn
* 3) A WallAnt takes no action each turn

A: 3

### Q3
Q: Choose the number of the correct choice:
* 0) Ant subclasses inherit the action method from the Ant class
* 1) Ant subclasses inherit the action method from the Insect class
* 2) Ant subclasses do not inherit the action method from any class

A: 1

### Q4
Q: If a subclass of Ant does not override the action method, what is the
default action?
Choose the number of the correct choice:
* 0) Move to the next place
* 1) Nothing
* 2) Reduce the health of all Bees in its place
* 3) Throw a leaf at the nearest Bee

A: 1

## Coding Solution

In [3]:
class WallAnt(Ant):
    name = 'Wall'
    food_cost = 4
    def __init__(self, health=4):
        """Create an Ant with a HEALTH quantity."""
        super().__init__(health)
    implemented = True   # Change to True to view in the GUI

# Probelm 7
## Unlock Questions
### Q1
Q: Should chew_timer be an instance or class attribute? Why?
Choose the number of the correct choice:
* 0) class, all HungryAnt instances in the game chew simultaneously
* 1) instance, all HungryAnt instances in the game chew simultaneously
* 2) instance, each HungryAnt instance chews independently of other
   HungryAnt instances
* 3) class, each HungryAnt instance chews independently of other
   HungryAnt instances
   
A: 2

### Q2
Q: When is a HungryAnt able to eat a Bee?
Choose the number of the correct choice:
* 0) Whenever a Bee is in its place
* 1) When it is chewing, i.e. when its chew_timer attribute is at least 1
* 2) Each turn
* 3) When it is not chewing, i.e. when its chew_timer attribute is 0

A: 3

### Q3
Q: When a HungryAnt is able to eat, which Bee does it eat?
Choose the number of the correct choice:
* 0) The closest Bee behind it
* 1) The closest Bee in either direction
* 2) A random Bee in the same place as itself
* 3) The closest Bee in front of it

A: 2

### Q4
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing HungryAnt parameters
>>> hungry = HungryAnt()
>>> HungryAnt.food_cost
```
A: 4

### Q5
```python
>>> hungry.health
```
A: 4

### Q6
```python
>>> hungry.time_to_chew
```
A: 3

### Q7
```python
>>> hungry.chew_timer
```
A: 0

### Q8
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing HungryAnt eats and chews
>>> hungry = HungryAnt()
>>> super_bee, wimpy_bee = Bee(1000), Bee(1)
>>> place = gamestate.places["tunnel_0_0"]
>>> place.add_insect(hungry)
>>> place.add_insect(super_bee)
>>> hungry.action(gamestate)         # super_bee is no match for HungryAnt!
>>> super_bee.health
```
A: 0

```python
>>> place.add_insect(wimpy_bee)
>>> for _ in range(3):
...     hungry.action(gamestate)     # chewing...not eating
>>> wimpy_bee.health
```
A: 1

```python
>>> hungry.action(gamestate)         # back to eating!
>>> wimpy_bee.health
```
A: 0

## Coding Solution

In [4]:
class HungryAnt(Ant):
    name = 'Hungry'
    time_to_chew = 3
    food_cost = 4
    def __init__(self):
        """Create an Ant with a HEALTH quantity."""
        super().__init__(health=1)
        self.chew_timer = 0
    
    def lucky_bee(self):
        if self.place.bees:
            return random_bee(self.place.bees)
        return None

    def swallow(self, target):
        if target is not None:
            damage = target.health if self.chew_timer == 0 else 0
            # print(damage)
            target.reduce_health(damage)
            self.chew_timer = self.time_to_chew
            
    def action(self, game_state):
        if self.chew_timer == 0:
            self.swallow(self.lucky_bee())
        else:
            self.chew_timer -= 1

    implemented = True   # Change to True to view in the GUI

# Problem 8
## Unlock Questions
### Q1
Q: Which Ant does a BodyguardAnt guard?
Choose the number of the correct choice:
* 0) All the Ant instances in the gamestate
* 1) The Ant instance in the place closest to its own place
* 2) The Ant instance that is in the same place as itself
* 3) A random Ant instance in the gamestate

A: 2

### Q2
Q: How does a BodyguardAnt guard its ant?
Choose the number of the correct choice:
* 0) By allowing Bees to pass without attacking
* 1) By attacking Bees that try to attack it
* 2) By increasing the ant's health
* 3) By protecting the ant from Bees and allowing it to perform its original action

A: 3

### Q3
Q: Where is the ant contained by a BodyguardAnt stored?
Choose the number of the correct choice:
* 0) In its place's ant instance attribute
* 1) Nowhere, a BodyguardAnt has no knowledge of the ant that it's protecting
* 2) In the BodyguardAnt's ant_contained instance attribute
* 3) In the BodyguardAnt's ant_contained class attribute

A: 2

### Q4
Choose the number of the correct choice:
* 0) When both Ant instances are containers
* 1) When exactly one of the Ant instances is a container and the
   container ant does not already contain another ant
* 2) There can never be two Ant instances in the same place
* 3) When exactly one of the Ant instances is a container

A: 1

### Q5
Q: If two Ants occupy the same Place, what is stored in that place's ant
instance attribute?
Choose the number of the correct choice:
* 0) The Ant being contained
* 1) Whichever Ant was placed there first
* 2) The Container Ant
* 3) A list containing both Ants

A: 2

### Q5
```python
>>> from ants import *
>>> # Testing BodyguardAnt parameters
>>> bodyguard = BodyguardAnt()
>>> BodyguardAnt.food_cost
```
A: 4

### Q6
```python
>>> bodyguard.health
```

A: 2


### Q7
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> gamestate = GameState(None, beehive, ant_types(), layout, (1, 9))
>>> #
>>> # Bodyguard ant added before another ant
>>> bodyguard = BodyguardAnt()
>>> other_ant = ThrowerAnt()
>>> place = gamestate.places['tunnel_0_0']
>>> place.add_insect(bodyguard)  # Bodyguard in place first
>>> place.add_insect(other_ant)
>>> place.ant is bodyguard
```
A: True


### Q8
```python
>>> bodyguard.ant_contained is other_ant
```
A: True


### Q9
```python
>>> from ants import *
>>> beehive, layout = Hive(AssaultPlan()), dry_layout
>>> gamestate = GameState(None, beehive, ant_types(), layout, (1, 9))
>>> #
>>> # Bodyguard ant can be added after another ant
>>> bodyguard = BodyguardAnt()
>>> other_ant = ThrowerAnt()
>>> place = gamestate.places['tunnel_0_0']
>>> place.add_insect(other_ant)  # Other ant in place first
>>> place.ant is other_ant
```
A True


### Q10
```python
>>> place.add_insect(bodyguard)
>>> place.ant is bodyguard
```
A: True
    
### Q11
```python    
>>> bodyguard.ant_contained is other_ant
```
A: True

**Note:** ```**kwargs``` and ```*args```

**1. \*args:** Let a function accept an arbitrary number of arguments

Sometimes, we would like to write a function that accepts an arbitrary number of arguments, and then calls another function using exactly those arguments.

Instead of listing formal parameters for a function, you can write ```*args```, which represents all of the arguments that get passed into the function. We can then call another function with these same arguments by passing these *args into this other function.

**Example1:**

In [5]:
def printx(f):
    def print_and_return(*args):
        result = f(*args)
        print('result: ', result)
        return result
    return print_and_return

In [6]:
def add3(x, y, z):
    return x + y + z

In [7]:
print_add3 = printx(add3)
print_add3(1, 2, 3)

result:  6


6

In [8]:
def add4(x, y, z, a):
    return x + y + z + a

In [9]:
print_add4 = printx(add4)
print_add4(1, 2, 3, 4)

result:  10


10

**2. **kwargs:** Let a function accept an arbitrary number of keyword arguments

In [10]:
def printk(f):
    def print_and_return(*args, **kwargs):
        result = f(*args, **kwargs)
        print('result: ', result)
        return result
    return print_and_return

In [11]:
def add1k(x, v1, v2):
    return x + v1 + v2

In [12]:
print_addk = printk(add1k)
print_addk(1, v1=2, v2=3)

result:  6


6

### Coding Solutions
#### 0. ```store_ant()``` and ```action()``` methods
* Set the instance attribute ```self.ant_contained``` to ```ant```
* If contains an ant, acts as the ```contained ant```

In [13]:
def store_ant(self, ant):
    # BEGIN Problem 8
    self.ant_contained = ant
    # END Problem 8
    
def action(self, gamestate):
    # BEGIN Problem 8
    if self.ant_contained:
        self.ant_contained.action(gamestate)
    # END Problem 8

#### 1. ```can_contain()``` method
* If ```container_ant``` contains **nothing** and ```other``` is **not** a ```container``` 

In [14]:
    def can_contain(self, other):
        # BEGIN Problem 8
        if self.ant_contained is None and not isinstance(other, ContainerAnt):
            return True
        else:
            return False
        # END Problem 8

#### 2. ```Ant.add_to()``` method
* Ant in place (```place.ant```), and we give this ant a ```containerAnt``` (```self.ant``` is a ```container```): 
    * **Note:** If two ants are in the same place, then place.ant should be set to refer to the container
    * First, call ```self.stores(other)```
    * Then, ```place.ant``` refers to the current ```containerAnt```
* Current ant in place is a containerAnt, we are adding ```self.ant``` into that ```container```:
    * Call ```place.stores(self)```
* Neither of the ants is a container:
    * **AssertionError** when attempting to put two ants in the same place

In [15]:
def add_to(self, place):
    if place.ant is None:
        place.ant = self
    else:
        # BEGIN Problem 8
        if self.can_contain(place.ant): # add container to the current place
            self.store_ant(place.ant)
            place.ant = self
        elif place.ant.can_contain(self): # add self to a container in place
            place.ant.store_ant(self)
        else:
            assert place.ant is None, 'Two ants in {0}'.format(place)
        # END Problem 8
    Insect.add_to(self, place)

# Problem 9

### Q1: 
Besides costing more to place, what is the only difference between a
TankAnt and a BodyguardAnt?
Choose the number of the correct choice:
* 0) A TankAnt increases the damage of the ant it contains
* 1) A TankAnt has greater health than a BodyguardAnt
* 2) A TankAnt does damage to all Bees in its place each turn
* 3) A TankAnt can contain multiple ants

A: 2


### Q2
```python
>>> from ants_plans import *
>>> from ants import *
>>> beehive, layout = Hive(make_test_assault_plan()), dry_layout
>>> dimensions = (1, 9)
>>> gamestate = GameState(None, beehive, ant_types(), layout, dimensions)
>>> #
>>> # Testing TankAnt parameters
>>> TankAnt.food_cost
```
A: 6

### Q3
```python
>>> TankAnt.damage
```
A: 1

### Q4
```python
>>> tank = TankAnt()
>>> tank.health
```
A: 2


## Coding Solutions

In [16]:
class TankAnt(ContainerAnt):
    name = 'Tank'
    food_cost = 6
    damage = 1
    
    def __init__(self, health=2):
        super().__init__(health)

    def action(self, gamestate):
        if self.ant_contained:
            self.ant_contained.action(gamestate)
        for bee in self.place.bees[:]:
            bee.reduce_health(self.damage)

    implemented = True